### Loading Libraries

In [1]:
from IPython.core.display import HTML, display
display(HTML("<style>.container { width:100% !important; }</style>"))
import numpy as np
import pandas as pd
import torch
import warnings
warnings.filterwarnings('ignore')
from sklearn.cross_validation import train_test_split
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier

path_to_infersent = "/Users/alvira/Desktop/squad/SQuAD/"
path = "../../practise/quora/"

/Users/alvira/anaconda2/envs/fastai/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Loading Dataset

In [3]:

train = pd.read_csv(path+"train.csv")
test = pd.read_csv(path+"test.csv")

In [4]:
train.shape, test.shape

((404290, 6), (2345796, 3))

In [5]:
subset = train.sample(50000, random_state=1).reset_index(drop=True)

### Creating Vocabulary

In [6]:
q1 = list(train["question1"])
q2 = list(train["question2"])
test_q1 = list(test["question1"])
test_q2 = list(test["question2"])

In [7]:
len(q1), len(q2), len(test_q1), len(test_q2)

(404290, 404290, 2345796, 2345796)

In [8]:
for item in q2: q1.append(item)
for item in test_q1: q1.append(item)
for item in test_q2: q1.append(item)

In [9]:
del q2, test_q1, test_q2

In [10]:
len(q1)

5500172

In [10]:
infersent = torch.load(path_to_infersent+'InferSent/encoder/infersent.allnli.pickle', map_location=lambda storage, loc: storage)
infersent.set_glove_path(path_to_infersent+"InferSent/dataset/GloVe/glove.840B.300d.txt")

In [14]:
infersent.build_vocab(q1[:350000], tokenize=True)

Found 74990(/93617) words with glove vectors
Vocab size : 74990


### Create Traning Set

In [15]:
train.shape[0]

404290

In [ ]:
trn = pd.DataFrame()
for i in range(train.shape[0]):
    print(i)
    diff = infersent.encode([train["question1"][i]], tokenize=True) - infersent.encode([train["question2"][i]], tokenize=True)
    trn  = pd.concat((trn, pd.Series(diff[0])), axis=1)

In [49]:
trn = trn.T

In [52]:
trn = trn.reset_index(drop=True)

In [53]:
trn.to_pickle("train")

In [11]:
trn = pd.read_pickle(path+"train")

In [12]:
y = train["is_duplicate"][:trn.shape[0]]

In [13]:
train_x, test_x, train_y, test_y = train_test_split(trn,
y, train_size=0.8, random_state = 5)

### Logistic Regression

In [14]:
lr = LogisticRegression()
lr.fit(train_x, train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [15]:
print("Multinomial Logistic regression Train Accuracy : ",accuracy_score(train_y, lr.predict(train_x)))
print("Multinomial Logistic regression Test Accuracy : ", accuracy_score(test_y, lr.predict(test_x)))

Multinomial Logistic regression Train Accuracy :  0.6778655244724044
Multinomial Logistic regression Test Accuracy :  0.5704853675945754


In [16]:
log_loss(test_y,lr.predict_proba(test_x)[:,1])

0.6780617789101567

### Random Forest

In [17]:
rf = RandomForestClassifier(max_leaf_nodes=50, n_estimators=50, min_samples_leaf=10)
rf.fit(train_x, train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=50,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=10, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [18]:
print("Multinomial Logistic regression Train Accuracy : ",accuracy_score(train_y, rf.predict(train_x)))
print("Multinomial Logistic regression Test Accuracy : ", accuracy_score(test_y, rf.predict(test_x)))

Multinomial Logistic regression Train Accuracy :  0.7277919064828448
Multinomial Logistic regression Test Accuracy :  0.7019985724482513


In [19]:
log_loss(test_y,rf.predict_proba(test_x)[:,1])

0.566607967460074

### XGBoost

In [ ]:
xg = GradientBoostingClassifier(min_samples_leaf=5,max_depth=10 )
xg.fit(train_x, train_y)

In [ ]:
print("Multinomial Logistic regression Train Accuracy : ",accuracy_score(train_y, xg.predict(train_x)))
print("Multinomial Logistic regression Test Accuracy : ", accuracy_score(test_y, xg.predict(test_x)))

In [ ]:
log_loss(test_y,xg.predict_proba(test_x)[:,1])